In [11]:
import os, sys

# Move one level up to Source Code
os.chdir(r"C:\Users\husseien\Desktop\340915149_322754953\Source Code")

# Optional: make sure Python can see this folder
sys.path.insert(0, os.getcwd())

print("CWD now:", os.getcwd())
print("sys.path[0]:", sys.path[0])
import pandas as pd
import os
import torch
import os, sys

# Add Source Code folder to sys.path
BASE_DIR = r"C:\Users\husseien\Desktop\340915149_322754953\Source Code"

if BASE_DIR not in sys.path:
    sys.path.insert(0, BASE_DIR)

# Optional: change working directory (recommended)
if os.getcwd() != BASE_DIR:
    os.chdir(BASE_DIR)

# Now you can import setup_paths
import setup_paths  # this will run the code in setup_paths.py


import os

# Base data folder
DATA_DIR = r"C:\Users\husseien\Desktop\340915149_322754953\Source Code\data"

# Path to labeled train.csv
TRAIN_CSV = os.path.join(DATA_DIR, "train.csv")

# Path to folder containing all raw CSV files
FILES_DIRECTORY = os.path.join(DATA_DIR, "unlabeled", "unlabeled")

# Check
print("TRAIN_CSV exists:", os.path.exists(TRAIN_CSV))
print("FILES_DIRECTORY exists:", os.path.exists(FILES_DIRECTORY))
print("Example CSVs:", os.listdir(FILES_DIRECTORY)[:5])


from CNN.CNN import MultivariateCNN
from models_utils.Datasets import *
from models_utils.GLOBALS import *
import torch.nn.functional as F

from CNN.cnn_utils import get_train_data, train_1D_cnn

CWD now: C:\Users\husseien\Desktop\340915149_322754953\Source Code
sys.path[0]: C:\Users\husseien\Desktop\340915149_322754953\Source Code
TRAIN_CSV exists: True
FILES_DIRECTORY exists: True
Example CSVs: ['0.csv', '1.csv', '10.csv', '100.csv', '1000.csv']


In [12]:
# get train data
train_data = pd.read_csv(TRAIN_CSV, index_col=0)
train_data['activity'] = train_data['activity'].map(activity_id_mapping)
data_type_1 = train_data[train_data['sensor'] == 'vicon'].reset_index()
data_type_2 = train_data[train_data['sensor'] == 'smartwatch'].reset_index()

In [13]:
# sizes of padding/cutting
target_size_type1 = 3000
target_size_type2 = 1169

In [14]:
# train or load models
train_or_load_autoencoders = 'load'
if train_or_load_autoencoders == 'train':
    model_CNN_type1 = train_1D_cnn('Type1Only1DCNN', data_type_1, '1', target_size_type1, 30, 64, 0.001)
    model_CNN_type2 = train_1D_cnn('Type2Only1DCNN', data_type_2, '2', target_size_type2, 20, 64, 0.001)
elif train_or_load_autoencoders == 'load':
    model_CNN_type1 = MultivariateCNN(3, target_size_type1, 18).to(device)
    model_CNN_type1.load_state_dict(torch.load('Type1Only1DCNN.pth'))
    model_CNN_type2 = MultivariateCNN(3, target_size_type2, 18).to(device)
    model_CNN_type2.load_state_dict(torch.load(f'Type2Only1DCNN.pth'))
else:
    raise ValueError('Wrong train or load')

RuntimeError: Error(s) in loading state_dict for MultivariateCNN:
	size mismatch for conv1.weight: copying a param with shape torch.Size([64, 1, 3]) from checkpoint, the shape in current model is torch.Size([64, 3, 3]).
	size mismatch for fc1.weight: copying a param with shape torch.Size([512, 288000]) from checkpoint, the shape in current model is torch.Size([512, 96000]).

In [ ]:
# calculate or load train_data
embedding_names = [f'embedding_feature_{i + 1}' for i in range(18)]
calculate_or_load_train_data = 'calculate'
if calculate_or_load_train_data == 'calculate':
    target_size_type1 = 3000
    target_size_type2 = 1169

    data_type_1, data_type_2 = get_train_data(
        'train_Only_CNN',
        model_CNN_type1,
        model_CNN_type2,
        target_size_type1,
        target_size_type2,
        embedding_size=18,
        is_autoencoder=False
    )

elif calculate_or_load_train_data == 'load':
    data_type_1 = pd.read_csv('train_Only_CNN_type1.csv')
    data_type_2 = pd.read_csv('train_Only_CNN_type2.csv')

RuntimeError: Given groups=1, weight of size [64, 1, 3], expected input[1, 3, 1169] to have 1 channels, but got 3 channels instead

In [ ]:
# get and save results for test data
results_list = []
sample_submission_path = os.path.join(DATA_DIR, "sample_submission.csv")

for i, file_id in enumerate(pd.read_csv(sample_submission_path)['sample_id'].to_list()):

    class_path = os.path.join(files_directory, f"{file_id}.csv")
    new_data = pd.read_csv(class_path)

    if new_data.shape[1] == 3:
        if len(new_data) < 4000:
            new_data = pad_sequence(new_data, 4000)
        new_data = torch.tensor(new_data.values, dtype=torch.float32).to(device)
        normalized_new_data = (new_data - min_values_type1) / (max_values_type1 - min_values_type1 + 1e-6)
        normalized_new_data = normalized_new_data.transpose(0, 1)
        logits = model_CNN_type1(normalized_new_data)


    else:
        new_data = new_data[new_data.iloc[:, 0] == 'acceleration [m/s/s]'].iloc[:, 1:]
        if len(new_data) < 1350:
            new_data = pad_sequence(new_data, 1350)
        new_data = torch.tensor(new_data.values, dtype=torch.float32).to(device)
        normalized_new_data = (new_data - min_values_type2) / (max_values_type2 - min_values_type2 + 1e-6)
        normalized_new_data = normalized_new_data.transpose(0, 1)

        logits = model_CNN_type2(normalized_new_data)

    predictions = F.softmax(logits, dim=1)
    res_dict = {activity: predictions.squeeze()[id].item() for id, activity in id_activity_mapping.items()}

    result_dict = {label: res_dict.get(label, 0) for label in activity_id_mapping.keys()}
    result_dict['sample_id'] = file_id
    results_list.append(result_dict)
results = pd.DataFrame(results_list, columns=['sample_id'] + list(activity_id_mapping.keys()))
results.fillna(0).to_csv('results_raw_cnn.csv', index=False)

RuntimeError: Given groups=1, weight of size [64, 1, 3], expected input[1, 3, 1350] to have 1 channels, but got 3 channels instead